In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot 


teams = pd.read_csv('MTeams.csv')
tourneystats = pd.read_csv('MNCAATourneyDetailedResults.csv')
MRegSeason = pd.read_csv('MRegularSeasonDetailedResults.csv')
PublicRanks = pd.read_csv('MMasseyOrdinals.csv')
Conferences = pd.read_csv('MTeamConferences.csv')
Coaches = pd.read_csv('MTeamCoaches.csv')
seeds = pd.read_csv('MNCAATourneySeeds.csv')



In [2]:
df = pd.concat(
    [
        MRegSeason[['Season','WTeamID', 'LTeamID', 'DayNum', 'WScore', 'LScore','NumOT','LFGM','WFGM','LFGA','WFGA','LFGM3','WFGM3','LFGA3','WFGA3','LFTM','WFTM','LFTA','WFTA','LOR','WOR','LDR','WDR','LAst','WAst','LTO','WTO','LStl','WStl','LBlk','WBlk','LPF', 'WPF']].assign(GameResult = 'W').rename(columns = {'WTeamID' : 'TeamID', 'LTeamID': 'OppTeamID', 'LScore' : 'OppScore', 'WScore': 'TeamScore',
        'WFGM': 'FGM', 'LFGM' : 'OppFGM','LFGA': 'OppFGA','WFGA': 'FGA','WFGM3' : 'FGM3', 'LFGM3': 'OppFGM3','WFGA3' :'FGA3', 'LFGA3' : 'OppFGA3','WFTM' : 'FTM', 'LFTM':'OppFTM','WFTA': 'FTA', 'LFTA' : 'OppFTA' ,'WOR' : 'OR', 'LOR' : 'OppOR','WDR': 'DR','LDR' : 'OppDR','WAst' : 'Ast', 'LAst' : 'OppAst','WTO': 'TO', 'LTO':'OppTO','WStl': 'Stl', 'LStl' : 'OppStl','WBlk': 'Blk', 'LBlk': 'OppBlk','WPF': 'PF', 'LPF' : 'OppPF'}),
        MRegSeason[['Season','WTeamID', 'LTeamID', 'DayNum', 'WScore', 'LScore','NumOT','LFGM','WFGM','LFGA','WFGA','LFGM3','WFGM3','LFGA3','WFGA3','LFTM','WFTM','LFTA','WFTA','LOR','WOR','LDR','WDR','LAst','WAst','LTO','WTO','LStl','WStl','LBlk','WBlk','LPF', 'WPF']].assign(GameResult = 'L').rename(columns = {'LTeamID' : 'TeamID', 'WTeamID': 'OppTeamID', 'LScore' : 'TeamScore', 'WScore': 'OppScore',
        'WFGM': 'OppFGM', 'LFGM' : 'FGM','LFGA': 'FGA','WFGA': 'OppFGA','WFGM3' : 'OppFGM3', 'LFGM3': 'FGM3','WFGA3' :'OppFGA3', 'LFGA3' : 'FGA3','WFTM' : 'OppFTM', 'LFTM':'FTM','WFTA': 'OppFTA', 'LFTA' : 'FTA' ,'WOR' : 'OppOR', 'LOR' : 'OR','WDR': 'OppDR','LDR' : 'DR','WAst' : 'OppAst', 'LAst' : 'Ast','WTO': 'OppTO', 'LTO':'TO','WStl': 'OppStl', 'LStl' : 'Stl','WBlk': 'OppBlk', 'LBlk': 'Blk','WPF': 'OppPF', 'LPF' : 'PF'})
    ]).reset_index(drop = True)



In [3]:
#Adding Variables
df['ScoreDiff'] = df['TeamScore'] - df['OppScore']
df['Win'] = (df['GameResult'] == 'W').astype('int') 



#maybe add assist to turnover ratio

df['FG%'] = df['FGM'] / df['FGA']
df['3PT%'] = df['FGM3'] / df['FGA3']
df['FT%'] = df['FTM'] / df['FTA']

In [4]:
agg_df = df.groupby(['Season','TeamID']).agg(
    MeanScoreDiff = ('ScoreDiff', 'mean'),
    MedianScoreDiff= ('ScoreDiff', 'median'),
    MinScoreDiff= ('ScoreDiff', 'min'),
    MaxScoreDiff= ('ScoreDiff', 'max'),
    MeanFG = ('FG%', 'mean'),
    Mean3PT = ('3PT%', 'mean'),
    MeanFT = ('FT%', 'mean'),
    MeanOR = ('OR', 'mean'),
    MeanDR = ('DR', 'mean'),
    MeanAst = ('Ast', 'mean'),
    MeanTO = ('TO', 'mean'),
    MeanStl = ('Stl', 'mean'),
    Meanblk = ('Blk', 'mean'),
    SumOT = ('NumOT', 'sum'),
    Wins = ("Win", "sum"), 
    Losses=('GameResult', lambda x: (x== "L").sum()),
    WinPercentage = ('Win', 'mean')
).reset_index()

In [5]:
seeds['OverallSeed'] = seeds['Seed'].str.replace('a', '').str.replace('b', "").str[1:].astype("int")

agg_df = agg_df.merge(
    seeds,on =['Season','TeamID'],how = 'left'
)

agg_df

,Season,TeamID,MeanScoreDiff,MedianScoreDiff,MinScoreDiff,MaxScoreDiff,MeanFG,Mean3PT,MeanFT,MeanOR,...,MeanAst,MeanTO,MeanStl,Meanblk,SumOT,Wins,Losses,WinPercentage,Seed,OverallSeed
0,2003,1102,0.250000,-3.0,-32,38,0.486149,0.367637,0.642402,4.178571,...,13.000000,11.428571,5.964286,1.785714,0,12,16,0.428571,NaN,NaN
1,2003,1103,0.629630,-2.0,-16,33,0.487294,0.331990,0.735271,9.777778,...,15.222222,12.629630,7.259259,2.333333,8,13,14,0.481481,NaN,NaN
2,2003,1104,4.285714,6.0,-19,28,0.419676,0.325442,0.705168,13.571429,...,12.107143,13.285714,6.607143,3.785714,1,17,11,0.607143,Y10,10.0
3,2003,1105,-4.884615,-3.5,-42,34,0.396204,0.359630,0.709598,13.500000,...,14.538462,18.653846,9.307692,2.076923,4,7,19,0.269231,NaN,NaN
4,2003,1106,-0.142857,-1.0,-26,26,0.425530,0.350196,0.623158,12.285714,...,11.678571,17.035714,8.357143,3.142857,1,13,15,0.464286,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7612,2024,1474,-5.928571,-4.5,-37,18,0.437888,0.328385,0.703711,8.428571,...,12.285714,11.428571,6.750000,2.785714,0,10,18,0.357143,NaN,NaN
7613,2024,1475,-8.115385,-7.5,-44,17,0.401983,0.325643,0.735838,8.538462,...,10.961538,12.884615,5.923077,2.576923,1,5,21,0.192308,NaN,NaN
7614,2024,1476,-13.571429,-10.5,-45,13,0.402395,0.312593,0.696320,6.714286,...,11.785714,13.250000,6.464286,2.285714,1,3,25,0.107143,NaN,NaN
7615,2024,1477,-10.520000,-10.0,-43,16,0.391687,0.301685,0.701378,7.120000,...,12.760000,11.200000,7.360000,4.720000,6,7,18,0.280000,NaN,NaN


In [6]:
#Doing the same as above with tourney data this time


 
winner_pov = tourneystats[['Season','WTeamID', 'LTeamID', 'DayNum', 'WScore', 'LScore','NumOT','LFGM','WFGM','LFGA','WFGA','LFGM3','WFGM3','LFGA3','WFGA3','LFTM','WFTM','LFTA','WFTA','LOR','WOR','LDR','WDR','LAst','WAst','LTO','WTO','LStl','WStl','LBlk','WBlk','LPF', 'WPF']].assign(GameResult = 'W').rename(columns = {'WTeamID' : 'TeamID', 'LTeamID': 'OppTeamID', 'LScore' : 'OppScore', 'WScore': 'TeamScore',
'WFGM': 'FGM', 'LFGM' : 'OppFGM','LFGA': 'OppFGA','WFGA': 'FGA','WFGM3' : 'FGM3', 'LFGM3': 'OppFGM3','WFGA3' :'FGA3', 'LFGA3' : 'OppFGA3','WFTM' : 'FTM', 'LFTM':'OppFTM','WFTA': 'FTA', 'LFTA' : 'OppFTA' ,'WOR' : 'OR', 'LOR' : 'OppOR','WDR': 'DR','LDR' : 'OppDR','WAst' : 'Ast', 'LAst' : 'OppAst','WTO': 'TO', 'LTO':'OppTO','WStl': 'Stl', 'LStl' : 'OppStl','WBlk': 'Blk', 'LBlk': 'OppBlk','WPF': 'PF', 'LPF' : 'OppPF'})

loser_pov = tourneystats[['Season','WTeamID','LTeamID', 'DayNum', 'WScore', 'LScore','NumOT','LFGM','WFGM','LFGA','WFGA','LFGM3','WFGM3','LFGA3','WFGA3','LFTM','WFTM','LFTA','WFTA','LOR','WOR','LDR','WDR','LAst','WAst','LTO','WTO','LStl','WStl','LBlk','WBlk','LPF', 'WPF']].assign(GameResult = 'L').rename(columns = {'LTeamID' : 'TeamID', 'WTeamID': 'OppTeamID', 'LScore' : 'TeamScore', 'WScore': 'OppScore',
'WFGM': 'OppFGM', 'LFGM' : 'FGM','LFGA': 'FGA','WFGA': 'OppFGA','WFGM3' : 'OppFGM3', 'LFGM3': 'FGM3','WFGA3' :'OppFGA3', 'LFGA3' : 'FGA3','WFTM' : 'OppFTM', 'LFTM':'FTM','WFTA': 'OppFTA', 'LFTA' : 'FTA' ,'WOR' : 'OppOR', 'LOR' : 'OR','WDR': 'OppDR','LDR' : 'DR','WAst' : 'OppAst', 'LAst' : 'Ast','WTO': 'OppTO', 'LTO':'TO','WStl': 'OppStl', 'LStl' : 'Stl','WBlk': 'OppBlk', 'LBlk': 'Blk','WPF': 'OppPF', 'LPF' : 'PF'})

full = pd.concat([winner_pov, loser_pov]).reset_index(drop=True)

full['ScoreDiff'] = full['TeamScore'] - full['OppScore']
full['Win'] = (full['GameResult'] == 'W').astype('int') 

full['FG%'] = full['FGM'] / full['FGA']
full['3PT%'] = full['FGM3'] / full['FGA3']
full['FT%'] = full['FTM'] / full['FTA']


full['Win'] = (full['GameResult'] == 'W').astype(
    "int"
)



seeds['OverallSeed'] = seeds['Seed'].str.replace('a', '').str.replace('b', "").str[1:].astype("int")

full_tourney = full.merge(
    seeds,on =['Season','TeamID'],how = 'left'
)

In [7]:
historic_tourney_stats = full_tourney.merge(
    agg_df[['Season', 'TeamID','WinPercentage', 'MedianScoreDiff','Mean3PT','MeanFG','MeanOR', 'MeanDR', 'Meanblk', 'MeanStl', 'MeanTO', 'MeanAst', 'MeanFT',  'OverallSeed']],
    on = ['Season', 'TeamID'],
    how='left',
    ).merge(
        agg_df[['Season',  'TeamID', 'WinPercentage', 'MedianScoreDiff','Mean3PT','MeanFG','MeanOR', 'MeanDR', 'Meanblk', 'MeanStl', 'MeanTO', 'MeanAst', 'MeanFT', 'OverallSeed']].rename(columns = {'TeamID':'OppTeamID', 'WinPercentage' : 'OppWinPercentage', 'MedianScoreDiff' : 'OppMedianScoreDiff','Mean3PT' : 'OppMean3PT', 'MeanFG' : 'OppMeanFG','MeanOR' : 'OppMeanOR', 'MeanDR' : 'OppMeanDR', 'Meanblk': 'OppMeanBlk', 'MeanStl': 'OppMeanStl', 'MeanTO' : 'OppMeanTO', 'MeanAst' : 'OppMeanAst', 'MeanFT' : 'OppMeanFT', 'OverallSeed' : 'OppOverallSeed'}
        ), on = ['Season',  'OppTeamID'],
    )
    



In [8]:
historic_tourney_stats['WinPctDiff'] = (historic_tourney_stats['WinPercentage'] - historic_tourney_stats['OppWinPercentage'])

historic_tourney_stats['Mean3PTDiff'] = (historic_tourney_stats['Mean3PT'] - historic_tourney_stats['OppMean3PT'])

historic_tourney_stats['MeanFGDiff'] = (historic_tourney_stats['MeanFG'] - historic_tourney_stats['OppMeanFG'])

historic_tourney_stats['MeanORDiff'] = (historic_tourney_stats['MeanOR'] - historic_tourney_stats['OppMeanOR'])

historic_tourney_stats['MeanDRDiff'] = (historic_tourney_stats['MeanDR'] - historic_tourney_stats['OppMeanDR'])

historic_tourney_stats['MeanBLKDiff'] = (historic_tourney_stats['Meanblk'] - historic_tourney_stats['OppMeanBlk'])

historic_tourney_stats['MeanStlDiff'] = (historic_tourney_stats['MeanStl'] - historic_tourney_stats['OppMeanStl'])

historic_tourney_stats['MeanTODiff'] = (historic_tourney_stats['MeanTO'] - historic_tourney_stats['OppMeanTO'])

historic_tourney_stats['MeanAstDiff'] = (historic_tourney_stats['MeanAst'] - historic_tourney_stats['OppMeanAst'])

historic_tourney_stats['MeanFTDiff'] = (historic_tourney_stats['MeanFT'] - historic_tourney_stats['OppMeanFT'])



In [9]:

teams = teams[['TeamID', 'TeamName']]

historic_tourney_stats = pd.merge(teams,historic_tourney_stats, on= 'TeamID', how = 'right')

historic_tourney_stats


,TeamID,TeamName,Season,OppTeamID,DayNum,TeamScore,OppScore,NumOT,OppFGM,FGM,...,WinPctDiff,Mean3PTDiff,MeanFGDiff,MeanORDiff,MeanDRDiff,MeanBLKDiff,MeanStlDiff,MeanTODiff,MeanAstDiff,MeanFTDiff
0,1421,UNC Asheville,2003,1411,134,92,84,1,29,32,...,-0.151724,0.042080,-0.016123,-0.890805,-1.627586,0.766667,0.635632,0.973563,-1.165517,0.152397
1,1112,Arizona,2003,1436,136,80,51,0,20,31,...,0.237685,-0.006859,0.017191,2.213054,1.918719,1.248768,1.602217,0.716749,3.435961,0.051446
2,1113,Arizona St,2003,1272,136,84,71,0,25,31,...,-0.172414,-0.015065,0.042222,-0.379310,-2.655172,-0.827586,-2.172414,0.206897,-1.068966,0.047369
3,1141,C Michigan,2003,1166,136,79,73,0,27,29,...,-0.085684,-0.007432,0.008041,-0.292581,0.094044,-0.454545,-1.290491,4.877743,-1.197492,0.073034
4,1143,California,2003,1301,136,76,74,1,25,27,...,0.124138,0.025371,0.010234,1.508046,2.345977,-0.273563,-1.214943,-0.027586,1.333333,-0.089516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2625,1388,St Mary's CA,2023,1163,139,55,70,0,27,19,...,0.023674,0.012188,-0.002961,-2.600379,-0.776515,-1.378788,0.235795,-2.864583,-4.835227,-0.054752
2626,1116,Arkansas,2023,1163,143,65,88,0,31,20,...,-0.151515,-0.033486,0.017376,-3.121212,-0.454545,0.363636,1.848485,0.181818,-3.848485,-0.063811
2627,1211,Gonzaga,2023,1163,145,54,82,0,30,20,...,0.086174,0.025019,0.060721,-2.850379,0.254735,-1.628788,0.704545,-2.145833,-1.053977,-0.045736
2628,1274,Miami FL,2023,1163,152,59,72,0,28,20,...,0.023674,0.020617,0.027329,-2.819129,-1.245265,-1.597538,0.798295,-1.395833,-2.335227,0.034994


In [10]:
teams.rename(columns={'TeamID' : 'OppTeamID', 'TeamName': 'OppTeamName' }, inplace = True)
historic_tourney_stats = pd.merge(teams,historic_tourney_stats, on = 'OppTeamID', how = 'right')
historic_tourney_stats

,OppTeamID,OppTeamName,TeamID,TeamName,Season,DayNum,TeamScore,OppScore,NumOT,OppFGM,...,WinPctDiff,Mean3PTDiff,MeanFGDiff,MeanORDiff,MeanDRDiff,MeanBLKDiff,MeanStlDiff,MeanTODiff,MeanAstDiff,MeanFTDiff
0,1411,TX Southern,1421,UNC Asheville,2003,134,92,84,1,29,...,-0.151724,0.042080,-0.016123,-0.890805,-1.627586,0.766667,0.635632,0.973563,-1.165517,0.152397
1,1436,Vermont,1112,Arizona,2003,136,80,51,0,20,...,0.237685,-0.006859,0.017191,2.213054,1.918719,1.248768,1.602217,0.716749,3.435961,0.051446
2,1272,Memphis,1113,Arizona St,2003,136,84,71,0,25,...,-0.172414,-0.015065,0.042222,-0.379310,-2.655172,-0.827586,-2.172414,0.206897,-1.068966,0.047369
3,1166,Creighton,1141,C Michigan,2003,136,79,73,0,27,...,-0.085684,-0.007432,0.008041,-0.292581,0.094044,-0.454545,-1.290491,4.877743,-1.197492,0.073034
4,1301,NC State,1143,California,2003,136,76,74,1,25,...,0.124138,0.025371,0.010234,1.508046,2.345977,-0.273563,-1.214943,-0.027586,1.333333,-0.089516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2625,1163,Connecticut,1388,St Mary's CA,2023,139,55,70,0,27,...,0.023674,0.012188,-0.002961,-2.600379,-0.776515,-1.378788,0.235795,-2.864583,-4.835227,-0.054752
2626,1163,Connecticut,1116,Arkansas,2023,143,65,88,0,31,...,-0.151515,-0.033486,0.017376,-3.121212,-0.454545,0.363636,1.848485,0.181818,-3.848485,-0.063811
2627,1163,Connecticut,1211,Gonzaga,2023,145,54,82,0,30,...,0.086174,0.025019,0.060721,-2.850379,0.254735,-1.628788,0.704545,-2.145833,-1.053977,-0.045736
2628,1163,Connecticut,1274,Miami FL,2023,152,59,72,0,28,...,0.023674,0.020617,0.027329,-2.819129,-1.245265,-1.597538,0.798295,-1.395833,-2.335227,0.034994


In [11]:

historic_tourney_stats.drop(columns={'OverallSeed_y', 'Seed'}, inplace = True)
historic_tourney_stats.rename(columns = {'OverallSeed_x': 'OverallSeed'}, inplace = True)

In [12]:
#Adding in 538 Scores to give us some more features 
stats_538_2024 = pd.read_csv('538forecast2024.csv',header=1)
stats_538_2023 = pd.read_csv('538stats2023.csv')
stats_538_2022 = pd.read_csv('538forecast2022.csv')
stats_538_2021 = pd.read_csv('538forecast2021.csv')
stats_538_2019 = pd.read_csv('538forecast2019.csv')
stats_538_2018 = pd.read_csv('538forecast2018.csv')
stats_538_2017 = pd.read_csv('538forecast2017.csv')
stats_538_2016 = pd.read_csv('538forecast2016.csv')

#stats_538_2024 = stats_538_2024[stats_538_2024['gender'] == 'mens']
stats_538_2023 = stats_538_2023[stats_538_2023['gender'] == 'mens']
stats_538_2022 = stats_538_2022[stats_538_2022['gender'] == 'mens']
stats_538_2021 = stats_538_2021[stats_538_2021['gender'] == 'mens']
stats_538_2019 = stats_538_2019[stats_538_2019['gender'] == 'mens']
stats_538_2018 = stats_538_2018[stats_538_2018['gender'] == 'mens']
stats_538_2017 = stats_538_2017[stats_538_2017['gender'] == 'mens']
stats_538_2016 = stats_538_2016[stats_538_2016['gender'] == 'mens']

In [13]:
stats_538_2024.rename(columns={'Team' : 'team_name', 'Composite' : 'team_rating'},inplace = True)
stats_538_2024['Season'] = 2024
stats_538_2024

,team_name,Seed,Region,Moore,Elo,LRMC,BPI,Pomeroy,S-Curve,Preseason,Unnamed: 10,team_rating,Injury adjustment,"Compoisite, after injuries",Season
0,Connecticut,1,East,95.4,97.5,93.7,95.0,96.4,98.3,91.5,NaN,95.6,0.0,95.6,2024
1,Purdue,1,Midwest,92.2,93.7,91.8,94.3,94.1,93.7,94.5,NaN,93.5,0.0,93.5,2024
2,Houston,1,South,95.4,93.7,95.2,98.0,96.1,95.2,90.8,NaN,94.9,-2.9,92.1,2024
3,Arizona,2,West,92.5,87.5,98.3,93.0,92.3,91.1,88.6,NaN,91.6,0.0,91.6,2024
4,Iowa State,2,East,93.7,94.2,90.0,92.7,92.3,90.0,81.4,NaN,91.0,-0.1,90.9,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,Montana State,16a,Midwest,68.8,73.8,69.7,65.8,69.5,73.0,76.5,NaN,71.1,0.0,71.1,2024
64,Stetson,16,East,70.5,71.3,69.5,71.4,69.3,73.6,69.7,NaN,70.7,0.1,70.8,2024
65,Howard,16a,West,64.4,69.9,68.1,67.2,66.2,71.6,71.5,NaN,68.4,0.0,68.4,2024
66,Grambling State,16b,Midwest,64.3,70.9,63.8,65.1,66.6,72.3,72.6,NaN,68.0,0.1,68.1,2024


In [14]:
stats_538_2024 = stats_538_2024[['Season', 'team_name', 'team_rating']]
stats_538_2024 = stats_538_2024.groupby('team_name').first().reset_index()


stats_538_2023['Season'] = pd.to_numeric(stats_538_2023['forecast_date'].str.slice(0, 4))
stats_538_2023 = stats_538_2023[['Season', 'team_name', 'team_rating']]
stats_538_2023 = stats_538_2023.groupby('team_name').first().reset_index()


stats_538_2022['Season'] = pd.to_numeric(stats_538_2022['forecast_date'].str.slice(0, 4))
stats_538_2022 = stats_538_2022[['Season', 'team_name', 'team_rating']]
stats_538_2022 = stats_538_2022.groupby('team_name').first().reset_index()


stats_538_2021['Season'] = pd.to_numeric(stats_538_2021['forecast_date'].str.slice(0, 4))
stats_538_2021 = stats_538_2021[['Season', 'team_name', 'team_rating']]
stats_538_2021 = stats_538_2021.groupby('team_name').first().reset_index()


stats_538_2019['Season'] = pd.to_numeric(stats_538_2019['forecast_date'].str.slice(0, 4))
stats_538_2019 = stats_538_2019[['Season', 'team_name', 'team_rating']]
stats_538_2019 = stats_538_2019.groupby('team_name').first().reset_index()


stats_538_2018['Season'] = pd.to_numeric(stats_538_2018['forecast_date'].str.slice(0, 4))
stats_538_2018 = stats_538_2018[['Season', 'team_name', 'team_rating']]
stats_538_2018 = stats_538_2018.groupby('team_name').first().reset_index()



stats_538_2017['Season'] = pd.to_numeric(stats_538_2017['forecast_date'].str.slice(0, 4))
stats_538_2017 = stats_538_2017[['Season', 'team_name', 'team_rating']]
stats_538_2017 = stats_538_2017.groupby('team_name').first().reset_index()


stats_538_2016['Season'] = pd.to_numeric(stats_538_2016['forecast_date'].str.slice(0, 4))
stats_538_2016 = stats_538_2016[['Season', 'team_name', 'team_rating']]
stats_538_2016 = stats_538_2016.groupby('team_name').first().reset_index()



stats_538 = pd.concat([stats_538_2016,stats_538_2017]).reset_index(drop=True)
stats_538 = pd.concat([stats_538,stats_538_2018]).reset_index(drop=True)
stats_538 = pd.concat([stats_538,stats_538_2019]).reset_index(drop=True)
stats_538 = pd.concat([stats_538,stats_538_2021]).reset_index(drop=True)
stats_538 = pd.concat([stats_538,stats_538_2022]).reset_index(drop=True)
stats_538 = pd.concat([stats_538,stats_538_2022]).reset_index(drop=True)
stats_538 = pd.concat([stats_538,stats_538_2023]).reset_index(drop=True)
stats_538 = pd.concat([stats_538,stats_538_2024]).reset_index(drop=True)


In [15]:
stats_538

,team_name,Season,team_rating
0,Arizona,2016,89.04
1,Arkansas-Little Rock,2016,78.90
2,Austin Peay,2016,68.83
3,Baylor,2016,85.47
4,Buffalo,2016,75.66
...,...,...,...
607,Wagner,2024,66.40
608,Washington State,2024,83.50
609,Western Kentucky,2024,74.70
610,Wisconsin,2024,86.60


In [16]:

stats_538.rename(columns={'team_name' : 'TeamName'},inplace = True)

In [17]:
stats_538

,TeamName,Season,team_rating
0,Arizona,2016,89.04
1,Arkansas-Little Rock,2016,78.90
2,Austin Peay,2016,68.83
3,Baylor,2016,85.47
4,Buffalo,2016,75.66
...,...,...,...
607,Wagner,2024,66.40
608,Washington State,2024,83.50
609,Western Kentucky,2024,74.70
610,Wisconsin,2024,86.60


In [18]:
historic_tourney_stats.columns

Index(['OppTeamID', 'OppTeamName', 'TeamID', 'TeamName', 'Season', 'DayNum',
       'TeamScore', 'OppScore', 'NumOT', 'OppFGM', 'FGM', 'OppFGA', 'FGA',
       'OppFGM3', 'FGM3', 'OppFGA3', 'FGA3', 'OppFTM', 'FTM', 'OppFTA', 'FTA',
       'OppOR', 'OR', 'OppDR', 'DR', 'OppAst', 'Ast', 'OppTO', 'TO', 'OppStl',
       'Stl', 'OppBlk', 'Blk', 'OppPF', 'PF', 'GameResult', 'ScoreDiff', 'Win',
       'FG%', '3PT%', 'FT%', 'OverallSeed', 'WinPercentage', 'MedianScoreDiff',
       'Mean3PT', 'MeanFG', 'MeanOR', 'MeanDR', 'Meanblk', 'MeanStl', 'MeanTO',
       'MeanAst', 'MeanFT', 'OppWinPercentage', 'OppMedianScoreDiff',
       'OppMean3PT', 'OppMeanFG', 'OppMeanOR', 'OppMeanDR', 'OppMeanBlk',
       'OppMeanStl', 'OppMeanTO', 'OppMeanAst', 'OppMeanFT', 'OppOverallSeed',
       'WinPctDiff', 'Mean3PTDiff', 'MeanFGDiff', 'MeanORDiff', 'MeanDRDiff',
       'MeanBLKDiff', 'MeanStlDiff', 'MeanTODiff', 'MeanAstDiff',
       'MeanFTDiff'],
      dtype='object')

In [19]:
simple_tourney = historic_tourney_stats[['TeamID', 'TeamName', 'OppTeamID', 'OppTeamName', 'Season', 'Win','WinPercentage', 'OppWinPercentage', 'MedianScoreDiff','OppMedianScoreDiff', 'OverallSeed', 'OppOverallSeed',
       'WinPctDiff', 'Mean3PTDiff', 'MeanFGDiff', 'MeanORDiff', 'MeanDRDiff',
       'MeanBLKDiff', 'MeanStlDiff', 'MeanTODiff', 'MeanAstDiff', 'MeanFTDiff']]



In [20]:
#Have to manually fix the spelling differences in team name between 538 and our data  

stats_538['TeamName'] = stats_538['TeamName'].str.replace('Texas Southern', 'TX Southern')
stats_538['TeamName'] = stats_538['TeamName'].str.replace('Southern California' , 'USC')
stats_538['TeamName'] = stats_538['TeamName'].str.replace('Kennesaw State' , 'Kennesaw')
stats_538['TeamName'] = stats_538['TeamName'].str.replace('Virginia Commonwealth' , 'VCU')
stats_538['TeamName'] = stats_538['TeamName'].str.replace('Cal State Bakersfield', 'CS Bakersfield')
stats_538['TeamName'] = stats_538['TeamName'].str.replace('Arkansas-Little Rock', 'Ark Little Rock')
stats_538['TeamName'] = stats_538['TeamName'].str.replace('Middle Tennessee', 'MTSU')
stats_538['TeamName'] = stats_538['TeamName'].str.replace('Stephen F. Austin', 'SF Austin')
stats_538['TeamName'] = stats_538['TeamName'].str.replace('Saint Joseph\'s', 'St Joseph\'s PA')
stats_538['TeamName'] = stats_538['TeamName'].str.replace('Green Bay', 'WI Green Bay')
stats_538['TeamName'] = stats_538['TeamName'].str.replace('UC-Davis', 'UC Davis')
stats_538['TeamName'] = stats_538['TeamName'].str.replace('North Carolina-Asheville', 'UNC Asheville')
stats_538['TeamName'] = stats_538['TeamName'].str.replace('Gardner-Webb', 'Gardner Webb')
stats_538['TeamName'] = stats_538['TeamName'].str.replace('State', 'St')
stats_538['TeamName'] = stats_538['TeamName'].str.replace('Saint Mary\'s (CA)', 'St Mary\'s CA')
stats_538['TeamName'] = stats_538['TeamName'].str.replace('Florida Gulf Coast', 'FL Gulf Coast')
stats_538['TeamName'] = stats_538['TeamName'].str.replace('Miami (FL)', 'Miami FL')
stats_538['TeamName'] = stats_538['TeamName'].str.replace('Fairleigh Dickinson', 'F Dickinson')

stats_538['TeamName'] = stats_538['TeamName'].str.replace('North Carolina-Wilmington' , 'UNC Wilmington')
stats_538['TeamName'] = stats_538['TeamName'].str.replace('Mount St. Mary\'s' , 'Mt St Mary\'s')
stats_538['TeamName'] = stats_538['TeamName'].str.replace('North Carolina Central' , 'NC Central')
stats_538['TeamName'] = stats_538['TeamName'].str.replace('Texas Christian' , 'TCU')
stats_538['TeamName'] = stats_538['TeamName'].str.replace('College of Charleston' , 'Col Charleston')
stats_538['TeamName'] = stats_538['TeamName'].str.replace('South Dakota St' , 'S Dakota St')
stats_538['TeamName'] = stats_538['TeamName'].str.replace('Kent St' , 'Kent')
stats_538['TeamName'] = stats_538['TeamName'].str.replace('Northern Kentucky' , 'N Kentucky')
stats_538['TeamName'] = stats_538['TeamName'].str.replace('Florida Atlantic' , 'FL Atlantic')
stats_538['TeamName'] = stats_538['TeamName'].str.replace('Southern Methodist' , 'SMU')
stats_538['TeamName'] = stats_538['TeamName'].str.replace('Florida Atlantic' , 'FL Atlantic')
stats_538['TeamName'] = stats_538['TeamName'].str.replace('East Tennessee St' , 'ETSU')
stats_538['TeamName'] = stats_538['TeamName'].str.replace('St. Bonaventure' , 'St Bonaventure')
stats_538['TeamName'] = stats_538['TeamName'].str.replace('North Carolina St' , 'NC State')
stats_538['TeamName'] = stats_538['TeamName'].str.replace('Loyola (IL)' , 'Loyola-Chicago')
stats_538['TeamName'] = stats_538['TeamName'].str.replace('Long Island University' , 'LIU Brooklyn')
stats_538['TeamName'] = stats_538['TeamName'].str.replace('Maryland-Baltimore County' , 'UMBC')
stats_538['TeamName'] = stats_538['TeamName'].str.replace('Southeast Missouri St' , 'SE Missouri St')
stats_538['TeamName'] = stats_538['TeamName'].str.replace('UC-Santa Barbara' , 'UC Santa Barbara')
stats_538['TeamName'] = stats_538['TeamName'].str.replace('Louisiana-Lafayette' , 'Louisiana')
stats_538['TeamName'] = stats_538['TeamName'].str.replace('Texas A&M-Corpus Christi', 'TAM C. Christi')
stats_538['TeamName'] = stats_538['TeamName'].str.replace('North Carolina-Greensboro', 'UNC Greensboro')
stats_538['TeamName'] = stats_538['TeamName'].str.replace('Cal St Fullerton', 'CS Fullerton')
stats_538['TeamName'] = stats_538['TeamName'].str.replace('Pennsylvania', 'Penn')
stats_538['TeamName'] = stats_538['TeamName'].str.replace('North Dakota St', 'N Dakota St')
stats_538['TeamName'] = stats_538['TeamName'].str.replace('Louisiana St', 'LSU')
stats_538['TeamName'] = stats_538['TeamName'].str.replace('St. John\'s (NY)', 'St John\'s')
stats_538['TeamName'] = stats_538['TeamName'].str.replace('UC-Irvine', 'UC Irvine')
stats_538['TeamName'] = stats_538['TeamName'].str.replace('Central Florida', 'UCF')
stats_538['TeamName'] = stats_538['TeamName'].str.replace('Saint Louis', 'St Louis')
stats_538['TeamName'] = stats_538['TeamName'].str.replace('Abilene Christian', 'Abilene Chr')
stats_538['TeamName'] = stats_538['TeamName'].str.replace('Eastern Washington', 'E Washington')
stats_538['TeamName'] = stats_538['TeamName'].str.replace('Brigham Young', 'BYU')
stats_538['TeamName'] = stats_538['TeamName'].str.replace('Saint Peter\'s', 'St Peter\'s')
stats_538['TeamName'] = stats_538['TeamName'].str.replace('Alabama-Birmingham', 'UAB')



simple_tourney = simple_tourney[simple_tourney['Season'] >= 2016]


full_tourney_data = simple_tourney.merge(stats_538, on=['TeamName', 'Season'], how = 'left').reset_index(drop=True)
#full_tourney_data = full_tourney_data.drop_duplicates().reset_index(drop=True)



full_tourney_data = full_tourney_data.merge(
    stats_538.rename(columns={'TeamName' : 'OppTeamName'}), on=['OppTeamName', 'Season'], how = 'left',suffixes=("","Opp")).reset_index(drop=True)
#full_tourney_data = full_tourney_data.drop_duplicates().reset_index(drop=True)


full_tourney_data['OppOverallSeed']  = full_tourney_data['OppOverallSeed'].astype(int)

In [21]:

full_tourney_data.loc[full_tourney_data['TeamName'] == 'Southern Univ','team_rating'] = 68.36
full_tourney_data.drop_duplicates(inplace= True)

In [22]:
#Adding some final features

full_tourney_data['rating_diff'] = (full_tourney_data['team_rating'] - full_tourney_data['team_ratingOpp'])
full_tourney_data['seed_diff'] = (full_tourney_data['OppOverallSeed'] - full_tourney_data['OverallSeed'])

In [23]:
#Setting a Baseline Prediction to see if our model works

full_tourney_data['BaselinePred'] = (full_tourney_data['OverallSeed'] < full_tourney_data['OppOverallSeed'])

full_tourney_data.loc[full_tourney_data['OverallSeed'] == full_tourney_data['OppOverallSeed'], 'BaselinePred',] = (full_tourney_data['WinPercentage'] > full_tourney_data['OppWinPercentage'])

full_tourney_data



,TeamID,TeamName,OppTeamID,OppTeamName,Season,Win,WinPercentage,OppWinPercentage,MedianScoreDiff,OppMedianScoreDiff,...,MeanBLKDiff,MeanStlDiff,MeanTODiff,MeanAstDiff,MeanFTDiff,team_rating,team_ratingOpp,rating_diff,seed_diff,BaselinePred
0,1195,FL Gulf Coast,1192,F Dickinson,2016,1,0.551724,0.548387,2.0,2.0,...,1.518354,-1.216908,-1.000000,-1.200222,-0.057349,71.41,66.72,4.69,0,True
1,1455,Wichita St,1435,Vanderbilt,2016,1,0.741935,0.593750,17.0,10.0,...,-1.950605,2.945565,-1.848790,-0.614919,0.020049,86.59,85.59,1.00,0,True
2,1221,Holy Cross,1380,Southern Univ,2016,1,0.424242,0.612903,-4.0,4.0,...,-0.807429,-0.900293,-0.352884,3.586510,0.011260,66.85,NaN,NaN,0,False
3,1276,Michigan,1409,Tulsa,2016,1,0.636364,0.645161,5.0,8.0,...,-0.586510,-1.382209,-1.079179,1.260020,0.046398,79.57,79.93,-0.36,0,False
4,1114,Ark Little Rock,1345,Purdue,2016,1,0.870968,0.764706,8.0,13.0,...,-1.582543,2.025617,-1.715370,-4.685009,-0.003942,78.90,88.68,-9.78,-7,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1333,1388,St Mary's CA,1163,Connecticut,2023,0,0.781250,0.757576,9.5,12.0,...,-1.378788,0.235795,-2.864583,-4.835227,-0.054752,87.00,96.43,-9.43,-1,False
1334,1116,Arkansas,1163,Connecticut,2023,0,0.606061,0.757576,6.0,12.0,...,0.363636,1.848485,0.181818,-3.848485,-0.063811,86.99,96.43,-9.44,-4,False
1335,1211,Gonzaga,1163,Connecticut,2023,0,0.843750,0.757576,10.0,12.0,...,-1.628788,0.704545,-2.145833,-1.053977,-0.045736,90.73,96.43,-5.70,1,True
1336,1274,Miami FL,1163,Connecticut,2023,0,0.781250,0.757576,7.0,12.0,...,-1.597538,0.798295,-1.395833,-2.335227,0.034994,90.44,96.43,-5.99,-1,False


In [24]:
#Very Basic, higher seed wins then those final #1 seeds compete based on winpercentage


from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss 

cv_scores_baseline = []
for season in full_tourney_data['Season'].unique():
    pred = full_tourney_data.query("Season == @season")[
        "BaselinePred"
    ].astype("int")
    y = full_tourney_data.query("Season == @season")['Win']
    score = accuracy_score(y,pred)
    score_ll = log_loss(y,pred)
    cv_scores_baseline.append(score)
    print(f"Holdout season {season} - Accuracy {score:0.4f} Log Loss {score_ll:0.4f}")


print(f'Baseline accuracy {np.mean(cv_scores_baseline):0.4f}')

Holdout season 2016 - Accuracy 0.6716 Log Loss 11.8352
Holdout season 2017 - Accuracy 0.7463 Log Loss 9.1454
Holdout season 2018 - Accuracy 0.6716 Log Loss 11.8352
Holdout season 2019 - Accuracy 0.6866 Log Loss 11.2973
Holdout season 2021 - Accuracy 0.6970 Log Loss 10.9223
Holdout season 2022 - Accuracy 0.6418 Log Loss 12.9112
Holdout season 2023 - Accuracy 0.7015 Log Loss 10.7593
Baseline accuracy 0.6881


In [32]:
#So Our model won't be worth more than picking the higher seed if we cant get an accuracy over 68%
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import cross_val_score
import xgboost as xgb

FEATURES = [
'WinPercentage',
'OppWinPercentage',
'MedianScoreDiff',
'OppMedianScoreDiff',
'Mean3PTDiff',
'MeanORDiff',
'MeanDRDiff',
'MeanFTDiff',
'MeanFGDiff',
'MeanTODiff',
'MeanAstDiff',
'MeanStlDiff',
'MeanBLKDiff',
'OverallSeed',
'OppOverallSeed',
'WinPctDiff',
'seed_diff',
'rating_diff',
'team_rating',
'team_ratingOpp'
]

TARGET = 'Win'


X= full_tourney_data[FEATURES]
y= full_tourney_data[TARGET]
groups = full_tourney_data['Season']
seasons = full_tourney_data['Season'].unique()



#cross validation with these seasons

kfold = GroupKFold(n_splits = full_tourney_data['Season'].nunique())
cv_results = []
models = []

season_idx = 0


for train_index, test_index in kfold.split(X,y,groups):
    X_train,X_test = X.iloc[train_index], X.iloc[test_index]
    y_train,y_test = y.iloc[train_index], y.iloc[test_index]
    
    #Making the model
    
    model = xgb.XGBRegressor(
        eval_metric ='logloss',
        n_estimators = 1_000,
        learning_rate = 0.001,
    )

    holdout_season = seasons[season_idx]
    print(f'Season left out: {holdout_season}') 

    #Training the model
    model.fit(X_train, y_train, eval_set = [(X_test, y_test)], verbose = 100)


    #predicting on test
    y_pred = model.predict(X_test)
    score_ll = log_loss(y_test,y_pred)
    y_pred = y_pred > 0.5

    #Evaluate model
    accuracy = accuracy_score(y_test,y_pred)
    cv_results.append(accuracy)
    season_idx += 1
    print(f'Season {holdout_season}: {accuracy} {score_ll}')
    models.append(model)


print('Average CV Accuracy:', np.mean(cv_results))

Season left out: 2016
[0]	validation_0-logloss:0.69261
[100]	validation_0-logloss:0.64451
[200]	validation_0-logloss:0.60442
[300]	validation_0-logloss:0.57078
[400]	validation_0-logloss:0.54247
[500]	validation_0-logloss:0.51852
[600]	validation_0-logloss:0.49764
[700]	validation_0-logloss:0.47907
[800]	validation_0-logloss:0.46289
[900]	validation_0-logloss:0.44757
[999]	validation_0-logloss:0.43228
Season 2016: 0.8432835820895522 0.4322760224243203
Season left out: 2017
[0]	validation_0-logloss:0.69273
[100]	validation_0-logloss:0.65475
[200]	validation_0-logloss:0.62313
[300]	validation_0-logloss:0.59910
[400]	validation_0-logloss:0.57988
[500]	validation_0-logloss:0.56281
[600]	validation_0-logloss:0.54909
[700]	validation_0-logloss:0.53722
[800]	validation_0-logloss:0.52806
[900]	validation_0-logloss:0.52079
[999]	validation_0-logloss:0.51556
Season 2017: 0.7835820895522388 0.5155593264838784
Season left out: 2018
[0]	validation_0-logloss:0.69267
[100]	validation_0-logloss:0.6494